## Experiments: Sensitive Analysis Hyperparameter RSAST:


It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 30 ,50 and 100 for the selected dataset.

In [3]:
#configure directory to import sast libraries
import sys 
import os 
#add sast library path
file_path = os.path.expanduser("~/random_sast/sast")
sys.path.append(file_path)
#add cd_diagram library path
file_path = os.path.expanduser("~/random_sast/cd_diagram")
sys.path.append(file_path)

#sys.path

In [4]:
from sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd
import os

### Reading Datasets

In [5]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsByClassifier'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        df['filename']=filename
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_other_methods = pd.concat(dfs, ignore_index=True)
df_other_methods=df_other_methods[['filename','folds:','0']]
df_other_methods['method']=df_other_methods['filename'].str.split('_').str[0]
df_other_methods
df_other_methods=df_other_methods.rename(columns={'folds:':'dataset','0':'score'})
df_other_methods=df_other_methods[['dataset','score','method']]


In [6]:
df_other_methods['method'].unique()

array(['BOSS', 'Catch22', 'cBOSS', 'HIVE-COTEv1', 'InceptionTime',
       'ProximityForest', 'ResNet', 'RISE', 'ROCKET', 'S-BOSS', 'SAST',
       'STC', 'TS-CHIEF', 'TSF', 'WEASEL'], dtype=object)

In [7]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsRsast'
#directory = os.getcwd()+'/results_accuracy_per_ds'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.startswith("df_overall_tunning") and filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        df['rpoint']=df['rpoint'].astype(str)
        df['nb_per_class']=df['nb_per_class'].astype(str)

        
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)
df_result.head(3)


,Unnamed: 0,accuracy,time,cweights_time,fsubsequence_time,tdataset_time,tclassifier_time,dataset_name,classifier_name,rpoint,nb_per_class,method,len_method
0,0,0.9,22.449903,0.118879,18.913046,3.344095,0.073366,BirdChicken,ACF&PACF: n_random_points=1 nb_inst_per_class=1,1,1,Rsast,ACF&PACF
1,1,0.9,0.423205,0.104959,0.011462,0.302871,0.003022,BirdChicken,ACF: n_random_points=1 nb_inst_per_class=1,1,1,Rsast,ACF
2,2,0.7,18.675146,0.104006,18.243167,0.323289,0.003877,BirdChicken,PACF: n_random_points=1 nb_inst_per_class=1,1,1,Rsast,PACF


In [8]:
#df_result=pd.read_csv("df_overall_tunning_SmoothSubspace3.csv")
df_result['dataset_name'].unique()


array(['BirdChicken', 'Chinatown', 'Coffee',
       'DistalPhalanxOutlineAgeGroup', 'DistalPhalanxOutlineCorrect',
       'GunPoint', 'ItalyPowerDemand', 'ShapeletSim', 'SmoothSubspace',
       'SonyAIBORobotSurface1', 'SonyAIBORobotSurface2',
       'SyntheticControl'], dtype=object)

In [9]:
df_result.columns.unique()

Index(['Unnamed: 0', 'accuracy', 'time', 'cweights_time', 'fsubsequence_time',
       'tdataset_time', 'tclassifier_time', 'dataset_name', 'classifier_name',
       'rpoint', 'nb_per_class', 'method', 'len_method'],
      dtype='object')

### Hyperparameter Tunning RSAST 

#### Accuracy: Subsequence Lenght Method 

In [10]:
# create a pivot table with the mean of score by dataset
len_method_vs_ds=pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['dataset_name'], aggfunc='mean')
len_method_vs_ds=np.transpose(len_method_vs_ds)#[['ACF','PACF']]
len_method_vs_ds=len_method_vs_ds.reset_index()
df_rocket=df_other_methods[df_other_methods["method"]=="STC"]
merged_df = len_method_vs_ds.merge(df_rocket,left_on='dataset_name', right_on='dataset',  how='left')
merged_df=merged_df.drop('method',axis=1)
merged_df

,dataset_name,ACF,ACF&PACF,Max ACF,Max PACF,None,PACF,dataset,score
0,BirdChicken,0.776667,0.830000,0.713333,0.755333,0.708667,0.837333,BirdChicken,0.950000
1,Chinatown,0.950865,0.962177,0.938503,0.960233,0.947833,0.963460,Chinatown,0.967930
2,Coffee,0.937143,0.997143,0.951429,0.937143,0.994286,0.992857,Coffee,1.000000
3,DistalPhalanxOutlineAgeGroup,0.613813,0.641823,0.559808,0.551367,0.543789,0.638082,DistalPhalanxOutlineAgeGroup,0.769784
4,DistalPhalanxOutlineCorrect,0.450097,0.433671,0.476039,0.452029,0.472850,0.454976,DistalPhalanxOutlineCorrect,0.771739
5,GunPoint,0.784519,0.972296,0.790296,0.950889,0.923185,0.966000,GunPoint,0.993333
6,ItalyPowerDemand,0.890897,0.903026,0.879585,0.874921,0.885079,0.897933,ItalyPowerDemand,0.955296
7,ShapeletSim,0.944296,0.891037,0.895704,0.734593,0.631259,0.881037,ShapeletSim,1.000000
8,SmoothSubspace,0.823500,0.868444,0.823556,0.857500,0.829389,0.869722,SmoothSubspace,0.933333
9,SonyAIBORobotSurface1,0.742696,0.789063,0.743738,0.722374,0.739567,0.789040,SonyAIBORobotSurface1,0.752080


In [11]:
# create a pivot table with the mean of score by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.796529,0.682221,0.751816,0.761918,0.745525,0.758425
ACF&PACF,0.899278,0.760498,0.785073,0.793820,0.786467,0.794332
Max ACF,0.796233,0.668181,0.735011,0.743333,0.717813,0.749829
Max PACF,0.876119,0.686611,0.755272,0.757954,0.738317,0.755694
None,0.841662,0.685108,0.733322,0.733541,0.722788,0.731447
PACF,0.898234,0.764183,0.778485,0.798713,0.796540,0.791933


In [12]:
# create a pivot table with the mean of score by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.796053,0.678066,0.736048,0.766310,0.755373,0.764297
ACF&PACF,0.901619,0.752473,0.773414,0.800047,0.795554,0.797775
Max ACF,0.788876,0.637460,0.735115,0.749056,0.739508,0.755937
Max PACF,0.884335,0.669519,0.757519,0.758198,0.746862,0.758502
None,0.846739,0.653664,0.731418,0.746978,0.729061,0.743077
PACF,0.901746,0.755895,0.775519,0.795842,0.803423,0.797785


#### Time: Subsequence Lenght Method 

In [13]:
# create a pivot table with the mean of the time by dataset
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['dataset_name'], aggfunc='mean')

dataset_name,BirdChicken,Chinatown,Coffee,DistalPhalanxOutlineAgeGroup,DistalPhalanxOutlineCorrect,GunPoint,ItalyPowerDemand,ShapeletSim,SmoothSubspace,SonyAIBORobotSurface1,SonyAIBORobotSurface2,SyntheticControl
len_method,,,,,,,,,,,,
ACF,81.630738,0.143859,29.515313,132.824072,177.888586,40.794863,0.826132,7.645123,1.599955,0.419210,0.815245,128.598058
ACF&PACF,297.280662,0.444094,102.533573,419.191777,519.712858,91.319471,2.032732,222.211859,2.648634,2.247746,3.008572,402.314136
Max ACF,2.745482,0.145839,2.160797,49.805655,59.238734,4.385836,0.839920,2.640422,1.595359,0.402863,0.627533,45.391469
Max PACF,107.885490,0.162721,6.330146,29.798286,35.902560,4.056074,0.915300,124.217464,1.849311,0.405555,0.495278,29.918500
None,1.853694,0.133453,1.236922,28.495936,34.720866,2.650650,0.829915,1.836391,1.571106,0.300002,0.397229,28.232846
PACF,208.928276,0.387296,74.365190,293.491555,345.791509,50.792711,2.043757,215.296044,2.606331,1.952652,2.348842,260.544206


In [14]:
# create a pivot table with the mean of the time by number of instances per class
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,6.188855,2.407699,23.840190,102.163468,49.856088,70.400397
ACF&PACF,13.589826,10.086542,95.880157,330.679404,177.452147,235.717335
Max ACF,1.264142,0.593156,5.081014,31.855638,13.571525,20.750537
Max PACF,1.197458,2.722645,26.588043,42.238120,35.783702,36.233985
None,0.906808,0.419194,3.202175,18.993520,8.319526,12.412691
PACF,8.271148,7.288207,72.246621,229.600723,124.239536,165.939948


In [15]:
# create a pivot table with the mean of the time by number of random points
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,9.124401,1.457653,13.217743,128.992997,39.030919,64.807578
ACF&PACF,20.236495,23.212640,58.140647,406.697713,137.590589,221.545369
Max ACF,1.366892,0.494400,3.887356,37.428506,11.228913,18.772059
Max PACF,1.160488,18.832087,20.404883,43.792161,27.902381,32.649605
None,0.867934,0.313622,2.366311,22.741683,6.836416,11.104447
PACF,11.743294,21.629675,44.528873,279.175615,100.686392,151.921316


#### Variance: Subsequence Lenght Method 

In [16]:
# create a pivot table with the variance of the accuracy by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.003930,0.041681,0.048382,0.042434,0.043519,0.043085
ACF&PACF,0.001382,0.045365,0.051599,0.045167,0.046390,0.047845
Max ACF,0.003911,0.049837,0.047591,0.048003,0.051042,0.045122
Max PACF,0.002207,0.058440,0.049458,0.053334,0.056982,0.054135
None,0.003414,0.053621,0.053523,0.051078,0.056629,0.050708
PACF,0.001377,0.041000,0.051067,0.047537,0.040474,0.046510


In [17]:
# create a pivot table with the variance of the accuracy by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='var')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.004839,0.050958,0.044549,0.039399,0.044474,0.040392
ACF&PACF,0.002834,0.051991,0.047186,0.044955,0.047223,0.045589
Max ACF,0.005164,0.050570,0.048155,0.044317,0.058709,0.044166
Max PACF,0.003802,0.063579,0.050270,0.049949,0.060968,0.050228
None,0.004996,0.061608,0.051738,0.048583,0.056067,0.051625
PACF,0.002360,0.045722,0.051593,0.040789,0.043185,0.045952


#### Another Results

In [18]:
df_result[['dataset_name','len_method','classifier_name']].describe()

,dataset_name,len_method,classifier_name
count,5730,5730,5730
unique,12,6,216
top,SmoothSubspace,ACF&PACF,None: n_random_points= (lenthg ts)//2 nb_inst_...
freq,720,955,35


In [19]:
pd.pivot_table(df_result, values='time', index=['dataset_name'], aggfunc='sum')/(60*60)

,time
dataset_name,
BirdChicken,14.590090
Chinatown,0.029526
Coffee,1.500986
DistalPhalanxOutlineAgeGroup,19.866818
DistalPhalanxOutlineCorrect,24.442815
GunPoint,4.849990
ItalyPowerDemand,0.155995
ShapeletSim,11.955152
SmoothSubspace,0.395690


In [20]:
df_result['time'].sum()/(60*60)

96.80863684124417

#### Generate CD Diagram Tunning Hyperparameter

In [21]:
#create dataset to generate cd diagram
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
#df_perf.classifier_name.unique()

In [22]:
from cd_function import *
#generate hyperparameter tuning cd diagrams
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF&PACF:')], labels=True, title="ACF&PACF comparison", name_file='images_cd_diagram/cd-diagram_ACF&PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^PACF:')], labels=True, title="PACF comparison", name_file='images_cd_diagram/cd-diagram_PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF:')], labels=True, title="ACF comparison", name_file='images_cd_diagram/cd-diagram_ACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^None:')], labels=True, title="None comparison", name_file='images_cd_diagram/cd-diagram_None.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max ACF:')], labels=True, title="Max ACF comparison", name_file='images_cd_diagram/cd-diagram_MaxACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max PACF:')], labels=True, title="Max PACF comparison", name_file='images_cd_diagram/cd-diagram_MaxPACF.png')
draw_cd_diagram(df_perf, labels=True, title="overall Hyperparameter Tunning", name_file='images_cd_diagram/cd-diagram_overall_tunning.png')



c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-val

In [23]:
_, average_ranks, _ = wilcoxon_holm(df_perf=df_perf)
min_rank= min(average_ranks)
average_ranks=pd.DataFrame(average_ranks)
average_ranks

c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


,0
Max PACF: n_random_points=30 nb_inst_per_class=1,127.916667
Max ACF: n_random_points=30 nb_inst_per_class=1,122.083333
Max ACF: n_random_points=1 nb_inst_per_class=1,121.583333
Max PACF: n_random_points=100 nb_inst_per_class=1,118.000000
Max ACF: n_random_points=10 nb_inst_per_class=1,117.541667
...,...
ACF&PACF: n_random_points=100 nb_inst_per_class=50,43.333333
ACF&PACF: n_random_points=100 nb_inst_per_class=30,42.958333
ACF&PACF: n_random_points=10 nb_inst_per_class=100,41.708333
ACF&PACF: n_random_points=50 nb_inst_per_class=100,39.416667


In [24]:
#generate example cd diagrams
draw_cd_diagram(pd.read_csv('example_cd_diagram.csv'), labels=True, title="Example CD Diagram", name_file='cd-diagram_example.png')

c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


#### Focus on most accurate lenght method

##### Overall Accuracy

In [25]:
# highest accurate hyperparameters
average_ranks[average_ranks[0]==min_rank][0].index

Index(['PACF: n_random_points=100 nb_inst_per_class=30'], dtype='object')

In [26]:
# filter by method with ACF&PACF
# create a pivot table with the mean of score by hyperparameter
df_result_acc=df_result[df_result["len_method"]=="ACF&PACF"]
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.906022,0.806883,0.884509,0.921422,0.920866,0.928988
1,0.883857,0.714988,0.696115,0.789771,0.790266,0.785953
10,0.905285,0.727182,0.790894,0.796666,0.795204,0.790671
100,0.905365,0.777466,0.786825,0.792186,0.795236,0.794422
30,0.888971,0.746645,0.782546,0.795352,0.793410,0.793275
50,0.902604,0.784883,0.787817,0.801271,0.777855,0.797541


In [27]:
# create a pivot table with the variatioon of score by hyperparameter
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.001153,0.008351,0.002187,0.001044,0.001114,0.000741
1,0.002133,0.062568,0.077627,0.040615,0.037238,0.046755
10,0.002339,0.059836,0.040519,0.047997,0.045891,0.050160
100,0.001299,0.035086,0.047333,0.052971,0.049867,0.051297
30,0.002060,0.046467,0.055026,0.048539,0.045650,0.051740
50,0.000833,0.031979,0.044719,0.047246,0.064878,0.051205


##### Overall time complexity

In [28]:
# create a pivot table with the mean of time spent in minutes
pd.pivot_table(df_result_acc, values=['time','cweights_time','fsubsequence_time','tdataset_time','tclassifier_time'], index=['dataset_name'], aggfunc='mean')/60

,cweights_time,fsubsequence_time,tclassifier_time,tdataset_time,time
dataset_name,,,,,
BirdChicken,0.001775,1.845793,0.000515,3.106501,4.954678
Chinatown,0.000160,0.001823,0.000126,0.005264,0.007402
Coffee,0.001159,0.119599,0.000517,1.587549,1.708893
DistalPhalanxOutlineAgeGroup,0.000686,0.038889,0.010279,6.936574,6.986530
DistalPhalanxOutlineCorrect,0.000844,0.028763,0.012983,8.619190,8.661881
GunPoint,0.000712,0.038966,0.001142,1.481085,1.521991
ItalyPowerDemand,0.000144,0.003278,0.000300,0.030122,0.033879
ShapeletSim,0.002136,2.069743,0.000366,1.631211,3.703531
SmoothSubspace,0.000103,0.002886,0.000560,0.040557,0.044144


In [29]:
# create a pivot table with the mean of time spent in minutes by random points and instances per class
pd.pivot_table(df_result_acc, values='time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')/60

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.282601,0.022526,0.232868,0.576652,0.563500,0.564199
1,0.010475,0.056340,0.424776,0.516456,0.507142,0.507167
10,0.049191,0.078892,0.720324,1.747770,1.123094,1.364348
100,0.434660,0.375743,3.697348,14.121683,6.960061,10.042684
30,0.140513,0.140927,1.359760,4.634129,2.447615,3.326647
50,0.217127,0.218616,2.068863,7.552542,4.242656,5.094940


##### Calculate weights time complexity

In [30]:
pd.pivot_table(df_result_acc, values='cweights_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.014897,0.014234,0.015946,0.016742,0.014963,0.015907
1,0.015020,0.039505,0.034749,0.043230,0.037325,0.038426
10,0.013960,0.041367,0.036154,0.046714,0.043347,0.042696
100,0.013454,0.052451,0.043583,0.046503,0.049913,0.050062
30,0.014453,0.044675,0.042425,0.049988,0.046137,0.050697
50,0.019168,0.046173,0.046191,0.052046,0.046225,0.050087


##### Finding subsequences time complexity

In [31]:
pd.pivot_table(df_result_acc, values='fsubsequence_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.242247,0.017882,0.223566,0.488185,0.435165,0.459952
1,0.211653,2.466725,23.405273,22.474001,26.292221,24.590836
10,0.222412,2.626947,23.550564,22.773997,26.564345,25.116368
100,0.278675,2.853488,27.140589,27.841491,25.001743,38.272257
30,0.236372,2.396395,22.864531,28.764379,27.377542,29.920977
50,0.266805,2.856334,28.456786,26.645855,30.253524,26.800147


##### Transform Dataset time complexity

In [32]:
pd.pivot_table(df_result_acc, values='tdataset_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,16.678743,1.311847,13.715811,34.062211,33.328868,33.344076
1,0.390414,0.851994,1.973064,8.379904,4.031182,5.722793
10,2.699057,2.011591,19.536340,81.891388,40.658658,56.582710
100,25.754204,19.530270,194.459079,817.589168,391.597881,562.884826
30,8.158566,5.938078,58.572563,247.579255,119.253728,169.298435
50,12.717282,10.107716,95.477171,408.686579,198.648432,278.487083


##### Classifier time complexity

In [33]:
pd.pivot_table(df_result_acc, values='tclassifier_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.016808,0.006692,0.014813,0.026530,0.027846,0.027762
1,0.010669,0.019162,0.072361,0.088792,0.066804,0.076676
10,0.014377,0.052410,0.095189,0.151160,0.117306,0.116650
100,0.030939,0.104079,0.191388,1.807877,0.944107,1.341054
30,0.018978,0.074968,0.103886,1.646955,0.174793,0.323106
50,0.021981,0.103675,0.147375,17.759222,25.605202,0.350316
